# Visualizations of the modified loss landscape $\mathbf{f_\lambda}$

This notebook is used for generating plots of how the (linearized) piecewise constant loss $f$ changes with hyperparameter $\lambda$. The method is proposed in [Differentiation of Blackbox Combinatorial Solvers](https://arxiv.org/abs/1912.02175). 

Implemented are solvers for **GraphMatching**, **MultiGraphMatching**, **TravellingSalesman**, **Ranking**, and **ShortestPath.**

The landscape is built by integrating the gradient calculated as $\nabla f_\lambda (w) = -\frac{1}{\lambda} [y(w) - y_\lambda (w)]$.

# Test iPyVolume plotting

Run to test whether ipyvolume installation succeeded (and all required jupyter extentions are enabled)

In [ ]:
import numpy as np
import ipyvolume as ipv

In [ ]:
V = np.zeros((128,128,128)) # our 3d array
# outer box
V[30:-30,30:-30,30:-30] = 0.75
V[35:-35,35:-35,35:-35] = 0.0
# inner box
V[50:-50,50:-50,50:-50] = 0.25
V[55:-55,55:-55,55:-55] = 0.0
ipv.quickvolshow(V, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)

# Loss landscape plots

Some example problems are generated in the following:

The magic constants are selected so that we obtain interesting sections of the high-dimensional function.

WARNING: It's 2D sections of high-D functions, sometimes may not be intuitive (interpolation happens in different dimension)

In [ ]:
from solvers_to_visualize import RankingBBSolver, TSPBBSolver, GraphMatchingBBSolver,  \
                                 ShortestPathBBSolver, MultiGraphMatchingBBSolver

### Ranking

In [ ]:
lambdas = [1e-6, 1, 2, 4, 8, 16]
rk_solv = RankingBBSolver(sequence_length=7, seed=45, w_normal_factor=30.0, w_shift_factor=20.0, y_addend=1)  # the last three arguments change the values of w and y_grad
rk_solv.plot_flambda(lambdas=lambdas, partitions=80, bounds1=(0.0, 1.0), bounds2=(0.0, 1.0), show_axes=False)

### TSP

In [ ]:
tsp_solv = TSPBBSolver(num_nodes=5, seed=42, w_normal_factor=10, w_shift_factor=20)
tsp_solv.plot_flambda(lambdas=lambdas, partitions=10, bounds1=(-1.0, 1.0), bounds2=(-1.0, 1.0), show_axes=False, show_box=False)

### Graph Matching

In [ ]:
lambdas = [1e-6, 10, 20, 40, 80, 160]
gm_solv = GraphMatchingBBSolver(num_nodes_l=[3, 3], num_edges_l=[2, 2], seed=43, w_normal_factor=10, y_factor=0.1)
gm_solv.plot_flambda(lambdas=lambdas, partitions=80, bounds1=(-1.0, 1.0), bounds2=(-1.0, 1.0), show_axes=False)

### Shortest path

In [ ]:
lambdas = [1e-6, 0.5, 1, 2, 4]
sp_solv = ShortestPathBBSolver(num_nodes=6, seed=30, w_normal_factor=10, y_factor=1, w_shift_addend=2)
sp_solv.plot_flambda(lambdas=lambdas, partitions=10, bounds1=(0.0, 1.0), bounds2=(0.0, 1.0), show_axes=False)

### Multigraph matching

In [ ]:
lambdas = [1e-6, 5, 10, 20, 40, 80]
mgm_solv = MultiGraphMatchingBBSolver(num_nodes_l=[3, 3], num_edges_l=[2, 2], seed=42, w_normal_factor=10, y_factor=0.1)
mgm_solv.plot_flambda(lambdas=lambdas, bounds1=(-2.0,2.0), bounds2=(-2.0,2.0), partitions=60, show_axes=False)

# Customize! Run your own solvers and cuts!

Problems can be added by specifying the solver and the input-generation function as shown below.
The input-generation should provide:
- the config for the solver
- a two-dimensional cut of a higher dimensional $w$-landscape 
- some gradient $\frac{dL}{dy}$ of the loss with respect to $y$, for which the "continuous interpolation" $f_\lambda$ should be plotted. 

For details on the provided helper functions to generate these inputs, as well as the calculations done for the plots, see utils.flamba_utils.

The challenge is always to find values for the $w$-cut and the $\frac{dL}{dy}$ that produce interesting results, as in producing multiple different outputs of the solver. To find these regions there are various parameters that can be changed, including shifts and factors for changing the randomized inputs to values that "fit" the solver, as well as the seed. The boundaries for the plotting region can also be changed.